# jfleg/correction

In [9]:
import pandas as pd
from utils import *
from tqdm import tqdm
from transformers import T5TokenizerFast, T5Model, T5Config
from torch.utils.data import DataLoader, ConcatDataset, WeightedRandomSampler, Subset
from Dataset import T5Dataset, JflegDataset
import os
import re
import pandas as pd
from torch.utils.data import Dataset
from transformers import T5TokenizerFast

In [16]:
with open("./jfleg/test/test.src") as fl:
    inputs = fl.readlines()
with open("./jfleg/test/test.ref0") as fl:
    ref0 = fl.readlines()
with open("./jfleg/test/test.ref1") as fl:
    ref1 = fl.readlines()
with open("./jfleg/test/test.ref2") as fl:
    ref2 = fl.readlines()
with open("./jfleg/test/test.ref3") as fl:
    ref3 = fl.readlines()

In [17]:
jfleg = pd.DataFrame({"inputs": inputs, "ref0": ref0, "ref1": ref1, "ref2":ref2, "ref3": ref3})

In [4]:
for col in jfleg.columns:
    jfleg[col] = jfleg[col].map(lambda x: x.replace('\n', ''))

In [5]:
jfleg["inputs"][0]

'New and new technology has been introduced to the society .'

In [6]:
jfleg.to_csv("./heptabot-train-data/jfleg-test.csv")

In [2]:
import torch 
import transformers 
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    

In [3]:

model = transformers.T5ForConditionalGeneration.from_pretrained("google/t5-v1_1-small")
path = "/home/nasorokin11/modelsbasebestlow2/student_74000"
model.load_state_dict(torch.load(path))
model.eval;

In [4]:
model.to(device);

In [5]:
teacher = transformers.T5ForConditionalGeneration.from_pretrained("google/t5-v1_1-base")
path = "/home/nasorokin11/modelsbasebestlow2/model-64844.checkpoint"
teacher.load_state_dict(torch.load(path))
teacher.to(device);
teacher.eval();

In [12]:
tokenizer = T5TokenizerFast.from_pretrained('/home/nasorokin11/T5small')
corr_dataset = T5Dataset("./heptabot-train-data/correction-eval.tsv", tokenizer, "correction:", target=1, source=0), [i for i in range(30)]
corr_dataloader = DataLoader(corr_dataset, batch_size=1, num_workers=4
                                 )

In [63]:
from utils import *

In [66]:
scores = 0
for batch in tqdm(corr_dataloader, "", len(corr_dataloader)):
    y = batch['target_ids'].to(device, dtype=torch.long)
    ids = batch['source_ids'].to(device, dtype=torch.long)
    mask = batch['source_mask'].to(device, dtype=torch.long)
    generated_ids = teacher.generate(
                                        input_ids=ids,
                                        attention_mask=mask,
                                        max_length=256,
                                    )
    preds, target, score = get_bleu(generated_ids, y)
    scores += score
    

 75%|███████▌  | 48/64 [06:19<02:06,  7.90s/it]


KeyboardInterrupt: 

In [ ]:
scores/len(corr_dataloader)

In [6]:
model_int8 = torch.quantization.quantize_dynamic(
    teacher.to("cpu"),  # the original model
    {torch.nn.Linear: torch.quantization.default_dynamic_qconfig},  # a set of layers to dynamically quantize
    dtype=torch.qint8)


In [7]:
path = os.path.join("/home/nasorokin11/final_models/T5-base-quantized", "model.checkpoint")
torch.save(model_int8.state_dict(), path)

In [20]:

tokenizer = T5TokenizerFast.from_pretrained('/home/nasorokin11/T5small')
jfleg_dataset = JflegDataset("./heptabot-train-data/jfleg-test.csv", tokenizer, "jfleg:", target=1, source=0)
jfleg_dataloader = DataLoader(jfleg_dataset, batch_size=128, num_workers=4
                                 )

In [18]:
def get_corpus_gleu_jfleg(pred, target, max_length=512):
    bleu = 0
    for i in range(pred.shape[0]):
        p = (tokenizer.decode(pred[i], skip_special_tokens=True)).split(" ")
        t = [(tokenizer.decode(target[i][j][0], skip_special_tokens=True)).split(" ") for j in range(len(target[i]))]
        bleu += sentence_gleu(t, p)
    return p, t, bleu/pred.shape[0]

In [12]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo):

In [21]:
scores = 0
for batch in tqdm(jfleg_dataloader, "", len(jfleg_dataloader)):
    ref_0 = batch['ref_0'].tolist()
    ref_1 = batch['ref_1'].tolist()
    ref_2 = batch['ref_2'].tolist()
    ref_3 = batch['ref_3'].tolist()
    ids = batch['source_ids'].to(device, dtype=torch.long)
    mask = batch['source_mask'].to(device, dtype=torch.long)
    generated_ids = model.generate(
                                        input_ids=ids,
                                        attention_mask=mask,
                                        max_length=256,
                                    )
    y = [[ref_0[i], ref_1[i], ref_2[i], ref_3[i]] for i in range(len(ref_0))]
    preds, target, score = get_corpus_gleu_jfleg(generated_ids, y)
    scores += score
    
    

100%|██████████| 6/6 [00:09<00:00,  1.52s/it]


In [22]:
scores/6

0.7796169800979033

In [34]:
tokenizer.decode(generated_ids[1], skip_special_tokens=True)

'One possible outcome is that an environmentally-induced reduction in motorization levels in the richer countries will outweigh any rise in motorization levels in the poorer countries.'

In [33]:
tokenizer.decode(ids[1], skip_special_tokens=True)

'jfleg: One possible outcome is that an environmentally-induced reduction in motorization levels in the richer countries will outweigh any rise in motorization levels in the poorer countries.'

In [36]:
tokenizer.decode(ref_0[1][0], skip_special_tokens=True)

'One possible outcome is that an environmentally-induced reduction in motorization levels in richer countries will outweigh any rise in motorization levels in poorer countries.'